In [1]:
import pandas as pd

In [2]:
cluster4 = pd.read_csv('cluster4.csv', index_col=0)
#cluster4.drop(cluster4.loc[1], inplace=True)
cluster4.reset_index(drop = True, inplace=True)
cluster4.head(15)

,Cluster Label,Zip Code,City,Community,Estimated Median Income,Longitude,Latitude,Total Population,Median Age,Total Males,Total Females,Total Households,Average Household Size,Factor
0,4,90011,Los Angeles,Los Angeles (Southeast Los Angeles),40940,-118.25868,34.007063,103892,26.2,52794,51098,22168,4.67,4253338480
1,4,90019,Los Angeles,"Los Angeles (Arlington Heights, Country Club P...",56389,-118.34015,34.048411,64458,35.8,31442,33016,23344,2.70,3634722162
2,4,90022,Los Angeles,East Los Angeles,46051,-118.15581,34.023662,67179,29.8,33216,33963,17023,3.94,3093660129
3,4,90026,Los Angeles,"Los Angeles (Echo Park, Silver Lake)",65269,-118.26596,34.078510,67869,34.0,34515,33354,24956,2.68,4429741761
4,4,90044,Los Angeles,"Athens, Los Angeles (South Los Angeles)",35981,-118.29158,33.953814,89779,28.6,43128,46651,25144,3.55,3230338199
5,4,90201,Bell,"Bell, Bell Gardens, Cudahy",44046,-118.17385,33.972814,101279,27.8,50658,50621,24104,4.16,4460934834
6,4,90250,Hawthorne,Hawthorne (Holly Park),56304,-118.35092,33.914614,93193,31.9,45113,48080,31087,2.98,5247138672
7,4,90255,Huntington Park,"Huntington Park, Walnut Park",44962,-118.21773,33.978404,75066,29.1,37525,37541,18419,4.06,3375117492
8,4,90262,Lynwood,Lynwood,52313,-118.20186,33.924642,69745,27.8,33919,35826,14669,4.57,3648570185
9,4,90280,South Gate,South Gate,52321,-118.19490,33.944264,94396,29.4,46321,48075,23278,4.05,4938893116


In [ ]:
import foursquare
import requests
CLIENT_ID = foursquare.CLIENT_ID
CLIENT_SECRET = foursquare.CLIENT_SECRET
VERSION = '20210501' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [4]:
record = cluster4.iloc[10]

In [43]:
import itertools
offset = 0
venues_list=[]

for _ in itertools.repeat(None, 100):
    url = 'https://api.foursquare' \
          '.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},' \
          '{}&radius={}&limit={}&offset={}&categoryId={}'.format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        record['Latitude'],
        record['Longitude'],
        4000,
        LIMIT,
        offset,
        '4d4b7105d754a06374d81259')

    # make the GET request
   # try:
    results = requests.get(url).json()["response"]['venues']
    for v in results:
        try:
            city = v['location']['city']
        except:
            city = 'N. A.'

        venues_list.append((
                    record['Zip Code'],
                    record['Community'],
                    record['Latitude'],
                    record['Longitude'],
                    v['name'],
                    # v['venue']['location']['lat'],
                    # v['venue']['location']['lng'],
                    v['categories'][0]['name'],
                    city ))
   # except:
   #     print('done')
    offset += 50

#nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
nearby_venues = pd.DataFrame(venues_list, columns=['Zip Code', 'Community',
              'Zip Code Latitude', 'Zip Code Longitude', 'Venue',
              # 'Venue Latitude',
              # 'Venue Longitude',
              'Venue Category',
              'City'])
nearby_venues.count()

Zip Code              5000
Community             5000
Zip Code Latitude     5000
Zip Code Longitude    5000
Venue                 5000
Venue Category        5000
City                  5000
dtype: int64

In [44]:
nearby_venues.count()

Zip Code              5000
Community             5000
Zip Code Latitude     5000
Zip Code Longitude    5000
Venue                 5000
Venue Category        5000
City                  5000
dtype: int64

In [45]:
nearby_venues.to_csv(path_or_buf='nearby_venues.csv')

In [40]:
ppds = nearby_venues.drop(nearby_venues[nearby_venues['City'] !=
                             'Norwalk'].index)

In [42]:
df = ppds[['Venue Category', 'Zip Code']]
df_grouped = df.groupby('Venue Category').agg(['mean', 'count'])
df_sorted = df_grouped.sort_values(by=[('Zip Code','count')], ascending=False)
#df_grouped.sort_values(by='count')
df_sorted

Zip Code      
                         mean count
Venue Category                     
Fast Food Restaurant    90650   800
Coffee Shop             90650   300
Grocery Store           90650   200
Convenience Store       90650   100
Donburi Restaurant      90650   100
Taco Place              90650   100

In [36]:
df = nearby_venues[['Venue Category']]
df_count = df.groupby('Venue Category').size().to_frame('Count').reset_index()
df_sorted = df_count.sort_values(by=[('Count')], ascending=False)
df_sorted

,Venue Category,Count
8,Fast Food Restaurant,1300
4,Coffee Shop,500
0,American Restaurant,400
14,Mexican Restaurant,400
12,Grocery Store,300
3,Bubble Tea Shop,200
10,Fried Chicken Joint,200
13,Korean Restaurant,200
15,Ramen Restaurant,100
21,Thai Restaurant,100


In [37]:
import plotly.express as px
fig = px.bar(df_sorted, x="Venue Category", y="Count", title='Distribution of Food Venues in Norwalk')
fig.show()

In [38]:
nearby_venues

,Zip Code,Community,Zip Code Latitude,Zip Code Longitude,Venue,Venue Category,City
0,90650,Norwalk,33.906914,-118.08339,Starbucks,Coffee Shop,Norwalk
1,90650,Norwalk,33.906914,-118.08339,McDonald's,Fast Food Restaurant,Norwalk
2,90650,Norwalk,33.906914,-118.08339,Del Taco,Fast Food Restaurant,Whittier
3,90650,Norwalk,33.906914,-118.08339,McDonald's,Fast Food Restaurant,Norwalk
4,90650,Norwalk,33.906914,-118.08339,7-Eleven,Convenience Store,Norwalk
...,...,...,...,...,...,...,...
4995,90650,Norwalk,33.906914,-118.08339,Lucky Thai,Thai Restaurant,Whittier
4996,90650,Norwalk,33.906914,-118.08339,Stater Bros. Markets,Grocery Store,Norwalk
4997,90650,Norwalk,33.906914,-118.08339,Taco Bell/Pizza Hut,Fast Food Restaurant,Norwalk
4998,90650,Norwalk,33.906914,-118.08339,McDonald's,Fast Food Restaurant,Norwalk


In [39]:
ppds

,Zip Code,Community,Zip Code Latitude,Zip Code Longitude,Venue,Venue Category,City
0,90650,Norwalk,33.906914,-118.08339,Starbucks,Coffee Shop,Norwalk
1,90650,Norwalk,33.906914,-118.08339,McDonald's,Fast Food Restaurant,Norwalk
3,90650,Norwalk,33.906914,-118.08339,McDonald's,Fast Food Restaurant,Norwalk
4,90650,Norwalk,33.906914,-118.08339,7-Eleven,Convenience Store,Norwalk
7,90650,Norwalk,33.906914,-118.08339,Tacos Gavilan,Taco Place,Norwalk
...,...,...,...,...,...,...,...
4993,90650,Norwalk,33.906914,-118.08339,Panera Bread,Coffee Shop,Norwalk
4996,90650,Norwalk,33.906914,-118.08339,Stater Bros. Markets,Grocery Store,Norwalk
4997,90650,Norwalk,33.906914,-118.08339,Taco Bell/Pizza Hut,Fast Food Restaurant,Norwalk
4998,90650,Norwalk,33.906914,-118.08339,McDonald's,Fast Food Restaurant,Norwalk
